In [108]:
import re
import pandas as pd
import sqlite3

# Parse logs

In [64]:
# define the search pattern
pattern = re.compile(r"(\d+-\d+-\d+ \d+:\d+:\d+,\d+:__main__:User \(\d+, \d+\) is protected)")

# list of logs to parse
logs = ['ignore/collection1.log', 'ignore/collection2.log', 'ignore/collection3.log']

# for each log, read in each line and return pattern matches
lines = []
for log in logs:
    for i, line in enumerate(open(log)):
        for match in re.finditer(pattern, line):
            lines.append(match.group())

# get the dates from returned lines
dates = [line[0:10] for line in lines]

# get the users from matched lines
user_pattern = re.compile(r"(?<=(User \())\d+, \d+(?=\) is protected)")
users = [re.search(user_pattern, line).group() for line in lines]

# split the two user IDs
users_1 = [user.split(', ')[0] for user in users]
users_2 = [user.split(', ')[1] for user in users]

# conver to a DF
df = pd.DataFrame({'date': dates, 'user1':users_1, 'user2':users_2})

# keep only the date and the first user ID
dates = df['date'].unique()
users = df['user1'].unique()

# conver to a data frame
fails = pd.DataFrame(index = users, columns = dates)

# set all values to zero
for col in fails.columns:
    fails[col].values[:] = 0

%%time
# impute values for failed pulls
for row in df.index:
    date = df.loc[row, 'date']
    user = df.loc[row, 'user1']
    fails.loc[user, date] = 1

# Flag bans

In [106]:
# define the window of days before and after the ban date
window = fails[fails['2021-01-09'] == 1].loc[:,'2021-01-01':'2021-01-14']

# users who were active for 5 consecutive days before 1/9, and inactive for 5 consecutive days after 1/9
banned = window[(window.loc[:,'2021-01-04':'2021-01-08'].sum(axis = 1) == 0) & (window.loc[:,'2021-01-09':'2021-01-14'].sum(axis = 1) == 5)]

# convert to a list of the banned users
banned = list(banned.index)
# convert to integers
banned = [int(user) for user in banned]

In [111]:
# pull data from the user database
conn = sqlite3.connect('../../OneDrive/SoDA 501/covid_twitter/users.db')
c = conn.cursor() 
c.execute("SELECT * FROM twitter_users")
users=c.fetchall()
conn.close()

users = pd.DataFrame(users, columns = ['user', 'location', 'city', 'state'])

In [129]:
# add the flag for banned users to the user data
banned_col = [1 if user in banned else 0 for user in users['user']]
users['banned'] = banned_col

# Add ideology score

In [139]:
# add ideology score to the user data
ideology = pd.read_csv('ideology_scores.csv')
ideology = ideology.filter(['ID', 'mean', 'Rhat'])
ideology.rename(columns = {'ID': 'user', 'mean':'ideology'}, inplace = True)

users = pd.merge(users, ideology, how = 'left', left_on = 'user', right_on ='user')

In [161]:
# update the database
conn = sqlite3.connect('../../OneDrive/SoDA 501/covid_twitter/users.db')
c = conn.cursor() 
c.execute("DROP TABLE twitter_users")
users.to_sql('twitter_users', con = conn, index = False)
c.execute("SELECT * FROM twitter_users")
test=c.fetchall()
conn.close()